In [ ]:
import jtrace
import re
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ibandstr = """
#
# Column 0: Name
# Column 1: Type
# Column 2: Curvature R (mm)
# Column 3: Thickness dz (mm)
# Column 4: Outer Radius (mm)
# Column 5: Inner Radius (mm)
# Column 6: Conic Constant Kappa
# Column 7 - 14: Aspheric Coefficient a_3 - a_10 (a_n r^n in meters)  #SIC.  THIS IS A LIE!
# Column 15: Coating file
# Column 16: Medium file
#
# (0)   (1)     (2)         (3)                 (4)     (5)     (6)     (7) (8) (9) (10)        (11) (12)       (13) (14)    (15)                (16)
M1      mirror  19835.0     0.0                 4180.0  2558.0  -1.215  0.0 0.0 0.0 1.381e-27   0.0  0.0        0.0  0.0     m1_protAl_Ideal.txt air
M2      mirror  6788.0      6156.2006           1710.0  900.0   -0.222  0.0 0.0 0.0 -1.274e-23  0.0  -9.68e-31  0.0  0.0     m2_protAl_Ideal.txt air
M3      mirror  8344.5      -6390.0006          2508.0  550.0   0.155   0.0 0.0 0.0 -4.5e-25    0.0  -8.15e-33  0.0  0.0     m3_protAl_Ideal.txt air
none    none    0.0         3630.5              0.0     0.0     0.0     0.0 0.0 0.0 0.0         0.0  0.0        0.0  0.0     none                air
L1      lens    2824.0      0.08213412779104    775.0   0.0     0.0     0.0 0.0 0.0 0.0         0.0  0.0        0.0  0.0     lenses.txt          silica_dispersion.txt
L1E     lens    5021.0      82.23               775.0   0.0     0.0     0.0 0.0 0.0 0.0         0.0  0.0        0.0  0.0     lenses.txt          air
L2      lens    0.0         412.64202           551.0   0.0     0.0     0.0 0.0 0.0 0.0         0.0  0.0        0.0  0.0     lenses.txt          silica_dispersion.txt
L2E    	lens    2529.0      30.0                551.0   0.0     -1.57   0.0 0.0 0.0 1.656e-21   0.0  0.0        0.0  0.0     lenses.txt          air
F       filter  5632.0      349.58              375.0   0.0     0.0     0.0 0.0 0.0 0.0         0.0  0.0        0.0  0.0     filter_3.txt        silica_dispersion.txt
FE      filter  5623.0      15.70               375.0   0.0     0.0     0.0 0.0 0.0 0.0         0.0  0.0        0.0  0.0     none                air
L3      lens    3169.0      53.30               361.0   0.0     -0.962  0.0 0.0 0.0 0.0         0.0  0.0        0.0  0.0     lenses.txt          silica_dispersion.txt
L3E     lens    -13360.0    60.0                361.0   0.0     0.0     0.0 0.0 0.0 0.0         0.0  0.0        0.0  0.0     lenses.txt          air
D       det     0.0         28.5                400.0   0.0     0.0     0.0 0.0 0.0 0.0         0.0  0.0        0.0  0.0     detectorar.txt      air
"""
re.sub("\s+"," ", ibandstr)
None

In [ ]:
air = jtrace.ConstMedium(1.000277)
w, n = np.genfromtxt("silica_dispersion.txt").T
w *= 1000  # microns -> nanometers
silica = jtrace.TableMedium(jtrace.Table(w, n, jtrace.Table.Interpolant.linear))

In [ ]:
def rays(theta_x, theta_y, wavelength):
    # Point towards (0,0,0), but at an angle.  Need to determine pupil locations.
    rs = np.linspace(telescope[0]['inner'], telescope[0]['outer'], 20)
    # The above works if theta is 0.
    # If theta is not zero, then need to shift the rays depending on how far away their origins are.
    # We'll set the z-origin of the rays to be 25 meters above the M1 vertex.
    z = 25
    dx = z * np.tan(theta_x)
    dy = z * np.tan(theta_y)
    rays_ = []
    for r in rs:        
        phis = np.linspace(0, 2*np.pi, int(128*r/telescope[0]['outer']), endpoint=False)
        for phi in phis:
            rays_.append(
                jtrace.Ray(jtrace.Vec3(r*np.cos(phi)+dx, r*np.sin(phi)+dy, z),
                           jtrace.Vec3(-np.tan(theta_x), -np.tan(theta_y), -1).UnitVec3()/air.getN(wavelength),
                           0, wavelength))
    return rays_

In [ ]:
def trace(ray, telescope):
    r = ray
    for optic in telescope:
        x0 = r.x0
        z0 = r.z0
        isec = optic['surface'].intersect(r)
        if optic['typ'] == 'mirror':
            r = isec.reflectedRay(r)
        elif optic['typ'] in ['lens', 'filter']:
            r = isec.refractedRay(r, optic['m0'], optic['m1'])
        x1 = isec.x0
        z1 = isec.z0
    return jtrace.Ray(r(isec.t), r.v, isec.t, r.wavelength)

def perturb(M1M3_dx=0, M1M3_dy=0, 
            M2_dx=0, M2_dy=0, M2_dz=0,
            cam_dx=0, cam_dy=0, cam_dz=0):
    telescope = []
    z = 0.0
    m0 = air
    m1 = silica
    for line in ibandstr.split('\n'):
        if len(line) == 0 : continue
        if line[0] == '#': continue
        name, typ, R, dz, outer, inner, kappa, a3, a4, a5, a6, a7, a8, a9, a10, coating, medium = line.split()
        z += float(dz)/1000
        if typ == 'none': continue
        m0 = m1
        if medium == 'air': 
            m1 = air
        else:
            m1 = silica
        if float(R) == 0:
            surface = jtrace.Plane(z)
        else:
            # Notice the manipulation of the raw aspheric coefficients below.  There's a negative sign missing, 
            # and also the coefficients are for mm instead of m, so there are factors of 1000^4, ^6, and ^8 missing
            # as well.
            surface = jtrace.Asphere(float(R)/1000,
                                     float(kappa), 
                                     [-float(a4)*10**(4*3), -float(a6)*10**(6*3), -float(a8)*10**(8*3)],
                                     z)
        if name in ['M1', 'M3']:
            surface = surface.shift(M1M3_dx, M1M3_dy, 0)
        elif name == 'M2':
            surface = surface.shift(M2_dx, M2_dy, M2_dz) 
        elif name[0] in ['L', 'F', 'D']:
            surface = surface.shift(cam_dx, cam_dy, cam_dz)
        telescope.append(dict(name=name, surface=surface, outer=float(outer)/1000, inner=float(inner)/1000, m0=m0, m1=m1, typ=typ))
    return telescope
telescope = perturb()

In [ ]:
@interact(wavelen=widgets.FloatSlider(min=300.0,max=1200.0,step=10.0,value=750.0),
          theta_x=widgets.FloatSlider(min=-1.75,max=1.75,step=0.01,value=1.75),
          theta_y=widgets.FloatSlider(min=-1.75,max=1.75,step=0.01,value=0.0),
          M1M3_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.00),
          M1M3_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dz=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dz=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          logscale=widgets.FloatSlider(min=1, max=3, step=0.1, value=1))
def spot(wavelen, theta_x, theta_y, M1M3_dx, M1M3_dy, M2_dx, M2_dy, M2_dz, cam_dx, cam_dy, cam_dz, logscale):
    """Display a spot diagram for LSST.

    @param wavelen  Wavelength in nm
    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param M1M3_dx  M1M3 x decenter in mm
    @param M1M3_dy  M1M3 y decenter in mm
    @param M2_dx    M2 x decenter in mm
    @param M2_dy    M2 y decenter in mm
    @param cam_dx   Camera x decenter in mm
    @param cam_dy   Camera y decenter in mm
    @param cam_dz   Camera z despace in mm
    @param logscale Logarithmic axes zoom level
    """
    telescope = perturb(M1M3_dx*1e-3, M1M3_dy*1e-3, 
                        M2_dx*1e-3, M2_dy*1e-3, M2_dz*1e-3,
                        cam_dx*1e-3, cam_dy*1e-3, cam_dz*1e-3)
    spots = []
    for ray in rays(theta_x*np.pi/180, theta_y*np.pi/180, wavelen):
        r = trace(ray, telescope)
        spots.append([r.p0.x, r.p0.y])
    spots = np.array(spots)
    spots -= np.mean(spots, axis=0)
    spots *= 1e6 # meters -> microns
    plt.figure(figsize=(4.5,4))
    plt.scatter(spots[:,0], spots[:,1], s=1, alpha=0.5)
    plt.xlim(-10**logscale, 10**logscale)
    plt.ylim(-10**logscale, 10**logscale)
    plt.title(r"$\theta_x = {:4.2f}\,,\theta_y = {:4.2f}$".format(theta_x, theta_y))
    plt.xlabel("microns")
    plt.ylabel("microns")

In [ ]:
#  http://stackoverflow.com/a/18968498
def planeFit(points):
    """
    p, n = planeFit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    from numpy.linalg import svd
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

In [ ]:
@interact(wavelen=widgets.FloatSlider(min=300.0,max=1200.0,step=10.0,value=750.0),
          theta_x=widgets.FloatSlider(min=-1.75,max=1.75,step=0.01,value=1.75),
          theta_y=widgets.FloatSlider(min=-1.75,max=1.75,step=0.01,value=0.0),
          M1M3_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.00),
          M1M3_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          M2_dz=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dz=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          logscale=widgets.FloatSlider(min=-6, max=-4, step=0.1, value=-5.5))
def opd(wavelen, theta_x, theta_y, M1M3_dx, M1M3_dy, M2_dx, M2_dy, M2_dz, cam_dx, cam_dy, cam_dz, logscale):
    """Display optical path differences

    @param wavelen  Wavelength in nm
    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param M1M3_dx  M1M3 x decenter in mm
    @param M1M3_dy  M1M3 y decenter in mm
    @param M2_dx    M2 x decenter in mm
    @param M2_dy    M2 y decenter in mm
    @param cam_dx   Camera x decenter in mm
    @param cam_dy   Camera y decenter in mm
    @param cam_dz   Camera z despace in mm
    @param logscale Logarithmic colorbar zoom level
    """
    telescope = perturb(M1M3_dx*1e-3, M1M3_dy*1e-3, 
                        M2_dx*1e-3, M2_dy*1e-3, M2_dz*1e-3,
                        cam_dx*1e-3, cam_dy*1e-3, cam_dz*1e-3)
    theta_opd = []
    for ray in rays(theta_x*np.pi/180, theta_y*np.pi/180, wavelen):
        r = trace(ray, telescope)
        theta_opd.append([r.v.x, r.v.y, r.t0])
    theta_opd = np.array(theta_opd)
    opd = theta_opd[:,2]
    opd[:] -= np.mean(opd)    
    x = theta_opd[:,0]
    y = theta_opd[:,1]
    p, n = planeFit(theta_opd[::10,:].T)
    const = np.dot(p, n)
    opd[:] -= (const-n[0]*x-n[1]*y)/n[2]
    plt.figure(figsize=(5.3,4))
    plt.scatter(x, y, c=opd, s=10, alpha=0.75, vmin=-10**logscale, vmax=10**logscale)
    plt.xlim(-0.6, 0.6)
    plt.ylim(-0.6, 0.6)
    plt.axhline(0.0, c='k')
    plt.axvline(0.0, c='k')
    plt.colorbar()